In [2]:
import java.util.List;
import java.util.ArrayList;
import java.util.LinkedList;
import java.util.Random;

public class ThreeDimensionalMaze {
	// order of dimension : [z][y][x]
	// so as to have xy planes in contiguoous memory locations for
	// connnectivity restrictions
	int[] start;
	int length= -1;
	int nbCarved= 0;
	boolean[][][] carved;
	int[][][][] nbCarvedNeighborsAndIdx;
	boolean [][][] wouldBreakPlanarConnectivity;
	List<int[]> withOneCarvedNeighbor = new ArrayList<int[]>();
	long seed;
	Random rng;

	boolean cannotBeCarved(int[] zyx) {
		int z= zyx[0];
		int y= zyx[1];
		int x= zyx[2];
		if( (z<= 0) || (z >= carved.length - 1) || (y <= 0) || (y >= carved[0].length - 1)
				|| (x <= 0) || (x >= carved[0][0].length - 1)
				|| carved[z][y][x]
				|| wouldBreakPlanarConnectivity[z][y][x]) {
			return true;
		}
		boolean result= false;
		try {
			carved[zyx[0]][zyx[1]][zyx[2]]= true;
			boolean[][] memoized= new boolean[carved[z].length][carved[z][y].length];
			boolean[][] connected= new boolean[carved[z].length][carved[z][y].length];

result=(!carved[z][y-1][x] && !isConnected(z,y-1,x, memoized, connected))
||(!carved[z][y+1][x] && !isConnected(z,y+1,x, memoized, connected))
||(!carved[z][y][x-1] && !isConnected(z,y,x-1, memoized, connected))
||(!carved[z][y][x+1] && !isConnected(z,y,x+1, memoized, connected));
		}finally {
			carved[zyx[0]][zyx[1]][zyx[2]]= false;
		}
		if(result) {
			wouldBreakPlanarConnectivity[zyx[0]][zyx[1]][zyx[2]]=true;
		}
		return result;
		// hasOnlyOneUncarvedNeighborOnXY is not enough
		// must check connectivity to the edges with a (depth-first) search from neighbors
		// after removing the cell
		// to 'memoize' the true values 
	}

	private boolean isConnected(int z, int y, int x, boolean[][] memoized, boolean[][] connected) {
		if((y <= 0) || (y >= carved[z].length-1) || (x <= 0) || (x >= carved[z][y].length-1)){
			return true;
		}
		if (!memoized[y][x]){
			memoized[y][x]= true;
			connected[y][x]=(!carved[z][y-1][x] && isConnected(z,y-1,x, memoized, connected))
			|| (!carved[z][y+1][x] && isConnected(z,y+1,x, memoized, connected))
			|| (!carved[z][y][x-1] && isConnected(z,y,x-1, memoized, connected))
			|| (!carved[z][y][x+1] && isConnected(z,y,x+1, memoized, connected));
			}
			return connected[y][x];

		
	}
	private boolean isDisconnectedFromPlanarEdge(int z, int y, int x, boolean[][][] visited) {
		if((y <= 0) || (y >= carved[z].length-1) || (x <= 0) || (x >= carved[z][y].length-1)){
			return false;
		}
		visited[z][y][x]= true;
		return  isDisconnectedFromPlanarEdge(z, y-1, x)
				&& isDisconnectedFromPlanarEdge(z, y+1, x)
				&& isDisconnectedFromPlanarEdge(z, y, x-1)
				&& isDisconnectedFromPlanarEdge(z, y, x+1);
	}
	private boolean isDisconnectedFromPlanarEdge(int z, int y, int x) {
		return isDisconnectedFromPlanarEdge(z, y, x, new boolean[carved.length][carved[0].length][carved[0][0].length]);
	}

	boolean hasOnlyOneUncarvedNeighborOnXY(int z, int y, int x) {
		boolean alreadyOne = (y > 0) && !carved[z][y - 1][x];
		if ((y < carved[z].length - 1) && !carved[z][y + 1][x]) {
			if (alreadyOne) {
				return false;
			}
			alreadyOne = true;
		}
		if ((x > 0) && !carved[z][y][x - 1]) {
			if (alreadyOne) {
				return false;
			}
			alreadyOne = true;
		}
		if ((x < carved[z][y].length - 1) && !carved[z][y][x + 1] && alreadyOne) {
			return false;
		}
		return true;
	}
	private boolean maybeUpdate(int z, int y, int x, int dist, int[][][] dists) {
		if((z<0) || (z >= dists.length) || (y<0) || (y >= dists[z].length)
				|| (x<0) || (x>= dists[z][y].length)|| !carved[z][y][x]
				||(dists[z][y][x]<= dist)) {
			return false;
		}
		dists[z][y][x]= dist;
		return true;
	}
	private boolean maybeUpdate(int z, int y, int x, int dist, int from, int[][][][] dists) {
		if((z<0) || (z >= dists.length) || (y<0) || (y >= dists[z].length)
				|| (x<0) || (x>= dists[z][y].length)|| !carved[z][y][x]
				||(dists[z][y][x][0]<= dist)) {
			return false;
		}
		dists[z][y][x][0]= dist;
		dists[z][y][x][1]= from;
		return true;
	}
	private static void dump(int[][][] dists) {
		for(int z=0; z != dists.length; ++z) {
			for(int y=0; y != dists[z].length; ++y) {
				for(int x=0; x != dists[z][y].length; ++x) {
					int d= dists[z][y][x];
					System.err.print(((d== Integer.MAX_VALUE) ? "__"
							:(d < 10) ? "0"+d: ""+d)+" ");
				}
				System.err.println();
			}
			System.err.println();
		}
	}
	int[][][] getDistFromStart(){
		return getDistFromStart(start);
	}
	int[][][] getDistFromStart(int[]start){
		int[][][] distFromStart= new int[carved.length][carved[0].length][carved[0][0].length];
		for(int iZ= 0; iZ != distFromStart.length; ++iZ) {
			for(int iY= 0; iY != distFromStart[iZ].length; ++iY) {
				for(int iX= 0; iX != distFromStart[iZ][iY].length; ++iX) {
					distFromStart[iZ][iY][iX]= Integer.MAX_VALUE;
				}
			}
		}
		distFromStart[start[0]][start[1]][start[2]]= 0;
		boolean updated= true;
		while(updated) {
			updated=false;
			for(int z= 0; z != distFromStart.length; ++z) {
				for(int y= 0; y != distFromStart[z].length; ++y) {
					for(int x= 0; x != distFromStart[z][y].length; ++x) {
						if(carved[z][y][x] && distFromStart[z][y][x] != Integer.MAX_VALUE) {
							int currentDist= distFromStart[z][y][x] +1;
							updated|= maybeUpdate(z-1, y, x, currentDist, distFromStart);
							updated|= maybeUpdate(z+1, y, x, currentDist, distFromStart);
							updated|= maybeUpdate(z, y-1, x, currentDist, distFromStart);
							updated|= maybeUpdate(z, y+1, x, currentDist, distFromStart);
							updated|= maybeUpdate(z, y, x-1, currentDist, distFromStart);
							updated|= maybeUpdate(z, y, x+1, currentDist, distFromStart);
						}
					}
				}
			}
		}
		return distFromStart;
	}
	int[][][][] getDistDirFromStart(int[]start){
		int[][][][] distFromStart= new int[carved.length][carved[0].length][carved[0][0].length][2];
		for(int iZ= 0; iZ != distFromStart.length; ++iZ) {
			for(int iY= 0; iY != distFromStart[iZ].length; ++iY) {
				for(int iX= 0; iX != distFromStart[iZ][iY].length; ++iX) {
					distFromStart[iZ][iY][iX]= new int[] {Integer.MAX_VALUE,-1};
				}
			}
		}
		distFromStart[start[0]][start[1]][start[2]][0]= 0;
		distFromStart[start[0]][start[1]][start[2]][0]= -1;
		boolean updated= true;
		while(updated) {
			updated=false;
			for(int z= 0; z != distFromStart.length; ++z) {
				for(int y= 0; y != distFromStart[z].length; ++y) {
					for(int x= 0; x != distFromStart[z][y].length; ++x) {
						if(carved[z][y][x] && distFromStart[z][y][x][0] != Integer.MAX_VALUE) {
							int currentDist= distFromStart[z][y][x][0] +1;
							updated|= maybeUpdate(z-1, y, x, currentDist,0, distFromStart);
							updated|= maybeUpdate(z+1, y, x, currentDist,1, distFromStart);
							updated|= maybeUpdate(z, y-1, x, currentDist,2, distFromStart);
							updated|= maybeUpdate(z, y+1, x, currentDist,3, distFromStart);
							updated|= maybeUpdate(z, y, x-1, currentDist,4, distFromStart);
							updated|= maybeUpdate(z, y, x+1, currentDist,5, distFromStart);
						}
					}
				}
			}
		}
		return distFromStart;
	}
	public List<int[]> getPath(){
		System.err.println("getPath from start "+toString(start));
		return getPath(start);
	}
	private void walkBackward(int[] zyx, int[][][][] distDirFromStart, int[] start) {
		try {
		switch(distDirFromStart[zyx[0]][zyx[1]][zyx[2]][1]) {
		case 0 :{
			++zyx[0];
			break;
		}
		case 1:{
			--zyx[0];
			break;
		}
		case 2:{
			++zyx[1];
			break;
		}
		case 3:{
			--zyx[1];
			break;
		}
		case 4:{
			++zyx[2];
			break;
		}
		case 5:{
			--zyx[2];
			break;
		}
		default :{
		}
		}
		}catch(Exception e) {
			zyx[0]= start[0];
			zyx[1]= start[1];
			zyx[2]= start[2];
		}
	}
	
	public List<int[]> getPath(int[] start){
		int[][][][] distDirFromStart=getDistDirFromStart(start);
		LinkedList<int[]> tmp= new LinkedList<int[]>();
		int count=0;
		for(int[] zyx= end(start, distDirFromStart); distDirFromStart[zyx[0]][zyx[1]][zyx[2]][0] !=-1; walkBackward(zyx, distDirFromStart,start)) {
			++count;
			System.err.println(toString(zyx)+" "+count);
			tmp.addFirst(zyx.clone());
		}
		tmp.addFirst(start);
		return new ArrayList<int[]>(tmp);
	}
	private int[] end(int[] start, int[][][][] distFromStart) {
		int[] res= {distFromStart.length-1, -1, -1};
		int yMiddle = distFromStart[res[0]].length/2;
		int xMiddle = distFromStart[res[0]][0].length/2;
		int length= -1;
		for(int y= 0; y != distFromStart[res[0]].length; ++y) {
			for(int x=0; x != distFromStart[res[0]][y].length; ++x) {
				if((carved[res[0]-1][y][x]) &&
						((length < distFromStart[res[0]-1][y][x][0])
						||((length == distFromStart[res[0]-1][y][x][0])
								&& (Math.abs(y-yMiddle)+Math.abs(x-xMiddle) <
										Math.abs(res[1]-yMiddle)+Math.abs(res[2]-xMiddle))))) {
					res[1]= y;
					res[2]= x;
					length= distFromStart[res[0]-1][y][x][0];				}
			}
		}
		++length;
		return res;
	}
	private int[] end(int[] start) {
		int[][][] distFromStart=getDistFromStart(start);
		//dump(distFromStart);
		int[] res= {distFromStart.length-1, -1, -1};
		int yMiddle = distFromStart[res[0]].length/2;
		int xMiddle = distFromStart[res[0]][0].length/2;
		length=-1;
		for(int y= 0; y != distFromStart[res[0]].length; ++y) {
			for(int x=0; x != distFromStart[res[0]][y].length; ++x) {
				if((carved[res[0]-1][y][x]) &&
						((length < distFromStart[res[0]-1][y][x])
						||((length == distFromStart[res[0]-1][y][x])
								&& (Math.abs(y-yMiddle)+Math.abs(x-xMiddle) <
										Math.abs(res[1]-yMiddle)+Math.abs(res[2]-xMiddle))))) {
					res[1]= y;
					res[2]= x;
					length= distFromStart[res[0]-1][y][x];				}
			}
		}
		++length;
		if(res[1]==-1) {
			System.err.println("!!!!!!!");
			int tmp=res[-1];
		}
		return res;
	}
	public ThreeDimensionalMaze(int[] coreDimensions, int[] start) {
	this(coreDimensions, start, System.nanoTime());
	}
	public ThreeDimensionalMaze(int[] coreDimensions, int[] start, long seed) {
		this.seed= seed;
		this.rng = new Random(seed);

		// should check that .length == 3 for all args
		this.start= start.clone();
		this.start[0]= 0; // force start on first level
		// add sides to the core
		carved = new boolean[coreDimensions[0] + 2][coreDimensions[1] + 2][coreDimensions[2] + 2];
		carved[this.start[0]][this.start[1]][this.start[2]] = true;

		wouldBreakPlanarConnectivity= new boolean[carved.length][carved[0].length][carved[0][0].length];
		nbCarvedNeighborsAndIdx = new int[carved.length][carved[0].length][carved[0][0].length][2];
		carve(this.start);
		carve();
	}

	public int carve() {
		for (int[] candidate = getCarvable(); candidate != null; candidate = getCarvable()) {
			if (!cannotBeCarved(candidate)) {
				carve(candidate);
			}
		}
		carve(end(start));
		return length;
	}

	private void carve(int[] zyx) {
		try {
		carved[zyx[0]][zyx[1]][zyx[2]] = true;
		}catch(Exception e) {
			System.err.println("error zyx ["+zyx[0]+","+zyx[1]+","+zyx[2]+"]");
		}
		++nbCarved;
		maybeAddCandidate(new int[] { zyx[0] - 1, zyx[1], zyx[2] });
		maybeAddCandidate(new int[] { zyx[0] + 1, zyx[1], zyx[2] });
		maybeAddCandidate(new int[] { zyx[0], zyx[1] - 1, zyx[2] });
		maybeAddCandidate(new int[] { zyx[0], zyx[1] + 1, zyx[2] });
		maybeAddCandidate(new int[] { zyx[0], zyx[1], zyx[2] - 1 });
		maybeAddCandidate(new int[] { zyx[0], zyx[1], zyx[2] + 1 });
	}
	static String toString(int[] zyx) {
		return "[" + zyx[0] + "][" + zyx[1] + "][" + zyx[2] + "]";
	}
	private static String toString(boolean[][] yxP) {
		StringBuilder res= new StringBuilder();
		for(int y= 0; y != yxP.length; ++y, res.append('\n')) {
			for(int x=0; x != yxP[y].length; ++x) {
				res.append(yxP[y][x] ? '⬜' : '⬛');
			}
		}
		return res.toString();
	}
	private void removeCandidateAt(int idx) {
		if (idx >= withOneCarvedNeighbor.size()) {
			System.err.println(" !!! idx:"+idx+" size:"+withOneCarvedNeighbor.size());
			return;
		}
		int[] zyx = withOneCarvedNeighbor.get(idx);
				if (idx != withOneCarvedNeighbor.size() - 1) {
			// replace with elt at the end for stability of indices (and efficient removal
			// from list)
			withOneCarvedNeighbor.set(idx, withOneCarvedNeighbor.get(withOneCarvedNeighbor.size() - 1));
			// have to update idx of the moved coords in nbCarvedNeighborsAndIdx
			/* int[] */ zyx = withOneCarvedNeighbor.get(idx);
			nbCarvedNeighborsAndIdx[zyx[0]][zyx[1]][zyx[2]][1] = idx;
		}
		withOneCarvedNeighbor.remove(withOneCarvedNeighbor.size() - 1);
	}

	private void maybeAddCandidate(int[] zyx) {
		// if nbCarvedNeighbors was one, remove if from list, if it was 0, add it
		if (cannotBeCarved(zyx)) {
			return;
		}
		nbCarvedNeighborsAndIdx[zyx[0]][zyx[1]][zyx[2]][0]++;
		switch (nbCarvedNeighborsAndIdx[zyx[0]][zyx[1]][zyx[2]][0]) {
		case 1: { // add to withOneCarvedNeighbor at the end
			nbCarvedNeighborsAndIdx[zyx[0]][zyx[1]][zyx[2]][1] = withOneCarvedNeighbor.size();
			withOneCarvedNeighbor.add(zyx);
			break;
		}
		case 2: { // remove from withOneCarvedNeighbor
			removeCandidateAt(nbCarvedNeighborsAndIdx[zyx[0]][zyx[1]][zyx[2]][1]);
			// nbCarvedNeighbors[zyx[0]][zyx[1]][zyx[2]][1]= -1; // not needed values is not
			// used if nb != 1
			break;
		}
		}
	}

	private int[] getCarvable() {
		if (withOneCarvedNeighbor.isEmpty()) {
			return null;
		}
		int idx = rng.nextInt(withOneCarvedNeighbor.size());
		int[] res = withOneCarvedNeighbor.get(idx);
		removeCandidateAt(idx);
		return res;
	}

	public String toString() {
		StringBuilder res = new StringBuilder();
		for (int iZ = 0; iZ != carved.length; ++iZ, res.append('\n')) {
			for (int iY = 0; iY != carved[iZ].length; ++iY, res.append('\n')) {
				for (int iX = 0; iX != carved[iZ][iY].length; ++iX) {
					res.append(carved[iZ][iY][iX] ? '⬜' : '⬛');
				}
			}
		}
		return res.toString();
	}
/*	private void updateNbCarved() {
		nbCarved= 0;
		for(int iZ= 0; iZ != carved.length; ++iZ) {
			for(int iY= 0; iY != carved[iZ].length; ++iY) {
				for(int iX= 0; iX != carved[iZ][iY].length; ++iX) {
					if(carved[iZ][iY][iX]) {
						++nbCarved;
					}
				}
			}
		}
	}
*/
	public static ThreeDimensionalMaze searchBest(int[] coreDimensions, int[] start, int nbTries) {
		ThreeDimensionalMaze res= new ThreeDimensionalMaze(coreDimensions, start);
		res.carve();
		for(int i=1; i != nbTries; ++i) {
			ThreeDimensionalMaze tmp= new ThreeDimensionalMaze(coreDimensions, start);
			tmp.carve();
//			System.err.println(tmp.seed);
//			System.err.println(tmp);
			if((tmp.length > res.length)||((tmp.length == res.length) && tmp.nbCarved > res.nbCarved)) {
				res= tmp;
			}
		}
		return res;
	}
	public static void main(String[] args) {
		//ThreeDimensionalMaze maze = searchBest(new int[] { 11, 11, 11 }, new int[] { 0, 6, 6 }, 10000000);
        ThreeDimensionalMaze maze = new ThreeDimensionalMaze(new int[] {11, 11, 11 }, new int[] { 0, 6, 6 }, 150571026368882L);

		System.out.println("\n"+maze);
		System.out.println("carved :"+maze.nbCarved+" out of :"+((maze.carved.length-2)*(maze.carved[0].length-2)*(maze.carved[0][0].length-2)));
		System.out.println("with path length:"+maze.length);
	}
}


com.twosigma.beaker.javash.bkrec691172.ThreeDimensionalMaze

In [3]:
%import com.twosigma.beakerx.widget.IntSlider

In [4]:
public class Controls {
public static final IntSlider w = new IntSlider();
public static final IntSlider h = new IntSlider();
public static final IntSlider n = new IntSlider();
    static{
w.setOrientation("horizontal");
w.setMax(20.0);
w.setMin(3.0);
h.setOrientation("horizontal");
h.setMax(20.0);
h.setMin(3.0);
n.setOrientation("horizontal");
n.setMax(20.0);
n.setMin(3.0);
    }
//w.orientation = "vertical"
}

com.twosigma.beaker.javash.bkr9b8e9fbe.Controls

In [6]:
%import com.twosigma.beakerx.widget.VBox

In [8]:
%import com.twosigma.beakerx.widget.Widget

In [11]:
return new VBox (java.util.Arrays.asList(Controls.w, Controls.h,Controls.n));

In [16]:
%import com.twosigma.beakerx.easyform.EasyForm

In [11]:
%%groovy
import com.twosigma.beakerx.widget.IntSlider
f = new EasyForm("Form and Run")
f.addTextField("first")
f['first'] = "First"
f.addTextField("last")
f['last'] = "Last"
w= new IntSlider()
w.setOrientation("horizontal");
w.setMax(20.0);
w.setMin(3.0);
f.addWidget("IntSlider",w)
f.addButton("Go!", "run")
f

In [19]:
%import com.twosigma.beakerx.easyform.EasyForm

In [20]:
EasyForm f= new EasyForm("Form and Run");

null

In [6]:
System.out.println("Hello !");

Hello !


null

In [7]:
import java.io.IOException;
import java.nio.file.Files;
import java.nio.file.Path;
import java.nio.file.Paths;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.HashSet;
import java.util.LinkedList;
import java.util.List;
import java.util.Set;

public class MazeToSVG {
    int w;
    int h;
    int thickness;
    int gridSize;
    int deltaX;
    int deltaY;
    double joinWidth;
    double kerf;
    double STROKE_WIDTH= 1;
    static final double JOIN_RADIUS=4.;
    private static final int ENTRY_RADIUS = 23;
    private static final int ENTRY_ARM_WIDTH = 10;
    private static final int ENTRY_GAP_RADIUS = 10;
    private static final int ENTRY_GAP_WIDTH = 20;
    private static final int ENTRY_ARM_RADIUS = 20;
    MazeToSVG(int w, int h, int thickness, int gridSize, int deltaX, int deltaY,double joinWidth, double kerf){
	this.w= w;
	this.h= h;
	this.thickness= thickness;
	this.gridSize= gridSize;
	this.deltaX= deltaX;
	this.deltaY= deltaY;
	this.joinWidth= joinWidth;
    }
    public static void main(String[] args) throws java.io.IOException {
	MazeToSVG converter= new MazeToSVG(1400,1400,50,50, 20, 20, 10, 3);
	//System.out.println(converter.oneWay(new int[] { 0, 3, 3 }, 100, 100));
		
	boolean f= false;
	boolean t= true;
	boolean[][] level=new boolean[][] {{f,f,f},{f,t,f},{f,f,f}};
	//System.out.println(converter.process(level));
	ThreeDimensionalMaze maze = new ThreeDimensionalMaze(new int[] {5, 5, 5 }, new int[] { 0, 3, 3 }, 215038569847684L);
	System.out.println(maze);	
	write("testing.svg",converter.process(maze));

    }
    public static void write(String filename, String content) throws IOException {	 
	Files.write(Paths.get(filename), content.getBytes());
	 
    }
    public  String process (ThreeDimensionalMaze maze) {
	int xOffset= deltaX;
	int yOffset= deltaY;
	StringBuilder res= new StringBuilder("<svg width=\""+w+"\" height=\""+h+"\" xmlns=\"http://www.w3.org/2000/svg\">\n");
	for(boolean[][] level : maze.carved) {
	    if(xOffset+level[0].length*gridSize > w) {
		xOffset= deltaX;
		yOffset+= level.length*gridSize+deltaY;
	    }
	    res.append(process(level,xOffset, yOffset));
	    for(int i=0; i !=4; ++i) {
		res.append(femaleJoinToSVGPath(femaleJoin(level, i),xOffset, yOffset));
	    }
	    xOffset+= level[0].length*gridSize+deltaX;
	}
	if(xOffset+maze.carved[0][0].length*gridSize > w) {
	    xOffset= deltaX;
	    yOffset+= maze.carved[0].length*gridSize+deltaY;
	}
	List<LinkedList<int[]>> tmp= new ArrayList<LinkedList<int[]>>();
	tmp.add(perimeter(maze.carved[0]));
	res.append(toString(tmp, xOffset, yOffset));
	res.append(oneWay(maze.start,xOffset, yOffset));
	for(int i=0; i !=4; ++i) {
	    res.append(femaleJoinToSVGPath(femaleJoin(maze.carved[0], i),xOffset, yOffset));
	}
	res.append("</svg>\n");
	return res.toString();
    }
    private static String toString(int[] xy) {
	return "["+xy[0]+","+xy[1]+"]";
    }
    private static List<LinkedList<int[]>>contours(Set<Segment> segs) {
	List<LinkedList<int[]>> lines= new ArrayList<LinkedList<int[]>>();
	for(Segment s : segs) {
	    LinkedList<int[]> tmp= new LinkedList<int[]>();
	    tmp.add(new int[] {s.x0, s.y0});
	    tmp.add(new int[] {s.x1, s.y1});
	    lines.add(tmp);
	}
	for(boolean couldMerge= true; couldMerge;) {
	    couldMerge= false;
	    for(int i=0; i < lines.size()-1; ++i) {
		LinkedList<int[]> current= lines.get(i);
		for(int j= i+1; j < lines.size();) {
		    boolean currentMerge= true;
		    int[] beg= current.get(0);
		    int[] end= current.get(current.size()-1);
		    LinkedList<int[]> candidate= lines.get(j);
		    if(Arrays.equals(candidate.get(0),end)) {
			for(int k=1; k !=candidate.size(); ++k) {// skip 0
			    current.add(candidate.get(k));
			}
		    }else if(Arrays.equals(candidate.get(0),beg)) { //BUG
			for(int k= 1; k != candidate.size(); ++k) {// skip 0
			    current.addFirst(candidate.get(k));
			}
		    }else if(Arrays.equals(candidate.get(candidate.size()-1), beg)) {
			for(int k= candidate.size()-2; k != -1; --k) {// skip last
			    current.addFirst(candidate.get(k));
			}
		    }
		    else if(Arrays.equals(candidate.get(candidate.size()-1), end)) {
			for(int k= candidate.size()-2; k != -1; --k) {// skip last
			    current.add(candidate.get(k));
			}
		    }else {
			currentMerge= false;
		    }
		    if(currentMerge) {
			lines.remove(j);
		    }else {
			++j;
		    }
		    couldMerge |=currentMerge;
		}
	    }
	}
	return lines;
    }
	
    private String femaleJoinToSVGPathCirc(List<double[]> line, int xOffset, int yOffset) {
	StringBuilder res= new StringBuilder("<polygon points=\"");
	for(double[] xy : line) {
	    res.append(""+(xy[0]+xOffset)+","+(xy[1]+yOffset)+" ");
	}
	res.append("\" style=\"fill:none;stroke:black;stroke-width:0.1\"/>\n");
	int count=0;
	String[] col= {"black", "red", "blue", "green"};
	for(double[] xy : line) {
	    res.append("<circle r=\""+JOIN_RADIUS+"\" cx=\""+(xy[0]+xOffset)+"\" cy=\""+(xy[1]+yOffset)
		       +"\" style=\"fill:none;stroke:"+col[count]+";stroke-width:1\"/>\n");
	    ++count;
	}
	return res.toString();
    }
    private String femaleJoinToSVGPath(List<double[]> line, int xOffset, int yOffset) {
	return"<path d=\""+
	    "M "+(line.get(0)[0]-JOIN_RADIUS+xOffset)+" "+(line.get(0)[1]+JOIN_RADIUS+yOffset)+"\n"
	    +"A "+JOIN_RADIUS+" "+JOIN_RADIUS+" 0 1 1 "
	    +(line.get(0)[0]+JOIN_RADIUS+xOffset)+" "+(line.get(0)[1]+JOIN_RADIUS+yOffset)+"\n"
	    +"L "+(line.get(1)[0]-JOIN_RADIUS+xOffset)+" "+(line.get(1)[1]-JOIN_RADIUS+yOffset)+"\n"
	    +"A "+JOIN_RADIUS+" "+JOIN_RADIUS+" 0 1 1 "
	    +(line.get(1)[0]-JOIN_RADIUS+xOffset)+" "+(line.get(1)[1]+JOIN_RADIUS+yOffset)+"\n"
	    +"L "+(line.get(2)[0]+JOIN_RADIUS+xOffset)+" "+(line.get(2)[1]-JOIN_RADIUS+yOffset)+"\n"
	    +"A "+JOIN_RADIUS+" "+JOIN_RADIUS+" 0 1 1 "
	    +(line.get(2)[0]-JOIN_RADIUS+xOffset)+" "+(line.get(2)[1]-JOIN_RADIUS+yOffset)+"\n"
	    +"L "+(line.get(3)[0]+JOIN_RADIUS+xOffset)+" "+(line.get(3)[1]+JOIN_RADIUS+yOffset)+"\n"
	    +"A "+JOIN_RADIUS+" "+JOIN_RADIUS+" 0 1 1 "
	    +(line.get(3)[0]+JOIN_RADIUS+xOffset)+" "+(line.get(3)[1]-JOIN_RADIUS+yOffset)+"\n"
	    +"L "+(line.get(0)[0]-JOIN_RADIUS+xOffset)+" "+(line.get(0)[1]+JOIN_RADIUS+yOffset)+"\""
	    +"\n style=\"fill:none;stroke:black;stroke-width:"+STROKE_WIDTH+"\"/>\n";
    }
    private String oneWay(int[] start, int xOffset, int yOffset) {
	StringBuilder res= new StringBuilder("<path d=\"");
	int centerY= (int)((start[1]+0.5)*gridSize+ yOffset);
	int centerX= (int)((start[2]+0.5)*gridSize+ xOffset);
	int x= centerX+ENTRY_RADIUS;
	res.append("M "+x+" "+centerY+"\n");
	x+= ENTRY_ARM_WIDTH;
	res.append("L "+x+" "+centerY+"\n");
	int y= centerY- ENTRY_RADIUS+ENTRY_GAP_RADIUS;
	res.append("L "+x+" "+y+"\n");
	x+= ENTRY_GAP_WIDTH;
	res.append("A "+ENTRY_GAP_RADIUS+" "+ENTRY_GAP_RADIUS+" 0 1 1 "+x+" "+y+"\n");
	y= centerY+ ENTRY_RADIUS;
	res.append("L "+x+" "+y+"\n");
	x= centerX+ENTRY_RADIUS;
	y= centerY+ENTRY_RADIUS+ENTRY_ARM_WIDTH+ENTRY_GAP_WIDTH;
	res.append("A "+ENTRY_GAP_RADIUS+" "+ENTRY_GAP_RADIUS+" 0 1 1 "+x+" "+y+"\n");
	x= centerX-ENTRY_RADIUS+ENTRY_GAP_RADIUS;
	res.append("L "+x+" "+y+"\n");
		
	y= centerY+ENTRY_RADIUS+ENTRY_ARM_WIDTH;
	res.append("A "+ ENTRY_GAP_RADIUS+" "+ENTRY_GAP_RADIUS+" 0 1 1 "+x+" "+y+"\n");
	x= centerX;
	res.append("L "+x+" "+y+"\n");
	y= centerY+ENTRY_RADIUS;
	res.append("L "+x+" "+y+"\n");
	x= centerX;
	res.append("L "+x+" "+y+"\n");
	x= centerX-ENTRY_RADIUS+ENTRY_ARM_RADIUS;
	res.append("L "+x+" "+y+"\n");
	x= centerX-ENTRY_RADIUS;
	y= centerY+ENTRY_RADIUS-ENTRY_ARM_RADIUS;
	res.append("A "+ENTRY_ARM_RADIUS+" "+ENTRY_ARM_RADIUS+" 0 0 1 "+x+" "+y+"\n");
	y= centerY;
	res.append("L "+x+" "+y+"\n");
	x-= ENTRY_ARM_WIDTH;
	res.append("L "+x+" "+y+"\n");
	y= centerY+ENTRY_RADIUS-ENTRY_GAP_RADIUS;
	res.append("L "+x+" "+y+"\n");
	x-= ENTRY_GAP_WIDTH;
	res.append("A "+ ENTRY_GAP_RADIUS+" "+ENTRY_GAP_RADIUS+" 0 1 1 "+x+" "+y+"\n");
	y= centerY-ENTRY_RADIUS;
	res.append("L "+x+" "+y+"\n");
	x= centerX-ENTRY_RADIUS;
	y= centerY-ENTRY_RADIUS-ENTRY_ARM_WIDTH-ENTRY_GAP_WIDTH;
	res.append("A "+ ENTRY_GAP_RADIUS+" "+ENTRY_GAP_RADIUS+" 0 1 1 "+x+" "+y+"\n");
	x= centerX+ENTRY_RADIUS-ENTRY_GAP_RADIUS;
	res.append("L "+x+" "+y+"\n");
	y+= ENTRY_GAP_WIDTH;
	res.append("A "+ ENTRY_GAP_RADIUS+" "+ENTRY_GAP_RADIUS+" 0 1 1 "+x+" "+y+"\n");
	x= centerX;
	res.append("L "+x+" "+y+"\n");
	y+= ENTRY_ARM_WIDTH;
	res.append("L "+x+" "+y+"\n");
	x= centerX+ENTRY_RADIUS-ENTRY_ARM_RADIUS;
	res.append("L "+x+" "+y+"\n");
	x= centerX+ENTRY_RADIUS;
	y=centerY-ENTRY_RADIUS+ENTRY_ARM_RADIUS;
	res.append("A "+ENTRY_ARM_RADIUS+" "+ENTRY_ARM_RADIUS+" 0 0 1 "+x+" "+y+"\n");
	res.append("Z\"  \n style=\"fill:none;stroke:black;stroke-width:"+STROKE_WIDTH+"\"/>\n/>");
	return res.toString();
    }
	
    static double[] add(double[] v0, double[] v1) {
	return new double[] {v0[0]+v1[0], v0[1]+v1[1]};
    }
    private List<double[]> femaleJoin( boolean[][] level, int kind){
	double[] center=new double[] {gridSize*3./4., gridSize*3./4.};
	double delta= thickness*Math.sqrt(2.)/4.;
	List<double []> res= new ArrayList<double[]>();
	switch (kind){
	case 0:{
	    double[] tmp= add(center, new double[] {delta, -delta});
	    res.add(add(tmp, new double[] {-joinWidth/2., -joinWidth/2} ));
	    res.add(add(tmp, new double[] {joinWidth/2., joinWidth/2} ));
	    tmp= add(center, new double[] {-delta, delta});
	    res.add(add(tmp, new double[] {joinWidth/2., joinWidth/2} ));
	    res.add(add(tmp, new double[] {-joinWidth/2., -joinWidth/2} ));
	    break;
	}
	case 1:{
	    center[0]= (level[0].length)*gridSize-center[0];
	    double[] tmp1= add(center, new double[] {-delta, -delta});
	    double[] tmp2= add(center, new double[] {delta, delta});
	    res.add(add(tmp1, new double[] {joinWidth/2., -joinWidth/2} ));
	    res.add(add(tmp2, new double[] {joinWidth/2., -joinWidth/2} ));
	    res.add(add(tmp2, new double[] {-joinWidth/2., joinWidth/2} ));
	    res.add(add(tmp1, new double[] {-joinWidth/2., +joinWidth/2} ));
	    break;
	}
	case 2:{
	    center[1]= (level.length)*gridSize-center[1];
	    double[] tmp1= add(center, new double[] {-delta, -delta});
	    double[] tmp2= add(center, new double[] {delta, delta});
	    res.add(add(tmp1, new double[] {joinWidth/2., -joinWidth/2} ));
	    res.add(add(tmp2, new double[] {joinWidth/2., -joinWidth/2} ));
	    res.add(add(tmp2, new double[] {-joinWidth/2., joinWidth/2} ));
	    res.add(add(tmp1, new double[] {-joinWidth/2., +joinWidth/2} ));
	    break;
	}
	case 3:{
	    center[0]= (level[0].length)*gridSize-center[0];
	    center[1]= (level.length)*gridSize-center[1];
	    double[] tmp= add(center, new double[] {delta, -delta});
	    res.add(add(tmp, new double[] {-joinWidth/2., -joinWidth/2} ));
	    res.add(add(tmp, new double[] {joinWidth/2., joinWidth/2} ));
	    tmp= add(center, new double[] {-delta, delta});
	    res.add(add(tmp, new double[] {joinWidth/2., joinWidth/2} ));
	    res.add(add(tmp, new double[] {-joinWidth/2., -joinWidth/2} ));
	    break;
	}
	default:
	    throw new IllegalArgumentException("kind "+kind +"unknown");
	}
	return res;
    }
    private static String toString(LinkedList<int[]> current) {
	StringBuilder res= new StringBuilder();
	for(int[] xy : current) {
	    res.append(toString(xy));
	}
	return res.toString();
    }
    public  String process (boolean[][] level, int xOffset, int yOffset) {
	Set<Segment> segs= new HashSet<Segment>();
	/* r,c:
	 * r,c     r,c+1
	 * r+1,c   r+1, c+1
	 * 
	 */
	for(int r=1; r != level.length-1; ++r) {
	    for(int c=1; c != level[r].length-1; ++c) {
		if(level[r][c]) {
		    if(!level[r][c-1]) {
			segs.add(new Segment(c*gridSize,r*gridSize, c*gridSize,  (r+1)*gridSize));
		    }
		    if(!level[r][c+1]) {
			segs.add(new Segment((c+1)*gridSize ,r*gridSize, (c+1)*gridSize,  (r+1)*gridSize));
		    }
		    if(!level[r-1][c]) {
			segs.add(new Segment(c*gridSize, r*gridSize, (c+1)*gridSize, r*gridSize ));
		    }
		    if(!level[r+1][c]) {
			segs.add(new Segment(c*gridSize, (r+1)*gridSize, (c+1)*gridSize, (r+1)*gridSize));
		    }
		}
	    }
	}
	List<LinkedList<int[]>> lines= contours(segs);
	/*
	  int levelH= level.length *gridSize;
	  int levelW= level[0].length*gridSize;
	  LinkedList<int[]> perimeter= new LinkedList<int[]>();
	  perimeter.add(new int[] {0,0});
	  perimeter.add(new int[] {0,levelW});
	  perimeter.add(new int[] {levelH,levelW});
	  perimeter.add(new int[] {levelH,0});
	  perimeter.add(new int[] {0,0});
	  lines.add(perimeter);
	*/
	lines.add(perimeter(level));
	return toString(lines, xOffset, yOffset);
    }
    LinkedList<int[]> perimeter(boolean[][] level){
	int levelH= level.length *gridSize;
	int levelW= level[0].length*gridSize;
	LinkedList<int[]> res= new LinkedList<int[]>();
	res.add(new int[] {0,0});
	res.add(new int[] {0,levelW});
	res.add(new int[] {levelH,levelW});
	res.add(new int[] {levelH,0});
	res.add(new int[] {0,0});
	return res;
    }
    String toString(List<LinkedList<int[]>> lines, int xOffset, int yOffset) {
	StringBuilder res= new StringBuilder("<g>\n");
	for(LinkedList<int[]> line : lines) {
	    StringBuilder current= new StringBuilder("<polyline points=\"");
	    for(int[] xy : line) {
		current.append(""+(xy[0]+xOffset)+","+(xy[1]+yOffset)+" ");
	    }
	    current.append("\" style=\"fill:none;stroke:black;stroke-width:"+STROKE_WIDTH+"\"/>\n");
	    res.append(current);
	}
	res.append("</g>\n");
	return res.toString();
    }

}

class Segment implements Comparable<Segment>{
    int x0;
    int y0;
    int x1;
    int y1;
    public Segment(int x0, int y0, int x1, int y1){
	if((x0 < x1) || ((x0 == x1)&& (y0 < y1))) {
	    this.x0= x0;
	    this.y0= y0;
	    this.x1= x1;
	    this.y1= y1;
	}else {
	    this.x0= x1;
	    this.y0= y1;
	    this.x1= x0;
	    this.y1= y0;
	}
	//System.err.println("constructed :"+x0+","+y0+" → "+x1+","+y1);
    }
    public boolean equals(Object o) {
	boolean res= false;
	if (o instanceof Segment) {
	    Segment s= (Segment) o;
	    res= (s.x0 == x0) && (s.y0 == y0)&&(s.x1 == x1)&&(s.y1 == y1);
	}
	return res;
    }
    private static int compare(int x, int y, int sx, int sy) {
	return 	 (x < sx)||((x == sx)&& (y< sy)) ? -1 : (((x == sx) && (y == sy))? 0 : 1);
    }
    public int compareTo(Segment s) {
	int res= compare(x0, y0, s.x0, s.y0);
	if (res==0) {
	    res= compare(x1, y1, s.x1, s.y1);
	}
	return res;
    }
    @Override
    public int hashCode() {
	return x0 ^y0 ^ x1 ^y1;
    }
}

com.twosigma.beaker.javash.bkrec691172.MazeToSVG

In [8]:
MazeToSVG.main(new String[0]);

⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬜⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛

⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬜⬜⬜⬛⬛
⬛⬜⬜⬛⬛⬛⬛
⬛⬛⬜⬜⬜⬜⬛
⬛⬛⬜⬛⬛⬜⬛
⬛⬜⬛⬛⬜⬜⬛
⬛⬛⬛⬛⬛⬛⬛

⬛⬛⬛⬛⬛⬛⬛
⬛⬜⬛⬛⬛⬜⬛
⬛⬛⬜⬛⬜⬛⬛
⬛⬛⬛⬜⬛⬛⬛
⬛⬜⬜⬛⬛⬛⬛
⬛⬜⬛⬜⬛⬜⬛
⬛⬛⬛⬛⬛⬛⬛

⬛⬛⬛⬛⬛⬛⬛
⬛⬜⬜⬜⬜⬜⬛
⬛⬛⬛⬛⬜⬛⬛
⬛⬜⬜⬛⬜⬜⬛
⬛⬜⬛⬛⬛⬜⬛
⬛⬛⬜⬜⬜⬛⬛
⬛⬛⬛⬛⬛⬛⬛

⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬜⬛⬜⬛⬛
⬛⬜⬜⬜⬛⬜⬛
⬛⬜⬛⬜⬛⬛⬛
⬛⬛⬛⬜⬛⬛⬛
⬛⬜⬜⬛⬜⬜⬛
⬛⬛⬛⬛⬛⬛⬛

⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬜⬛⬛⬜⬛
⬛⬜⬛⬜⬜⬜⬛
⬛⬛⬛⬛⬛⬜⬛
⬛⬜⬜⬛⬜⬜⬛
⬛⬜⬛⬜⬜⬛⬛
⬛⬛⬛⬛⬛⬛⬛

⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬜⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛




null